<a href="https://colab.research.google.com/github/nachofranco17/ProyectoCNN/blob/main/ProyectoCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install kagglehub
import kagglehub
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Descargando el dataset
path = kagglehub.dataset_download("nih-chest-xrays/data")
print("Dataset downloaded to:", path)

# Cargando el CSV
csv_path = os.path.join(path, "Data_Entry_2017.csv")
df = pd.read_csv(csv_path)
print("CSV loaded:", df.shape)
print(df.head())

In [ ]:
# Mostrando algunas imagenes

image_folder = os.path.join(path, "images")
print("Number of images:", len(os.listdir(image_folder)))
print("Example file:", os.listdir(image_folder)[:5])

img_path = os.path.join(image_folder, os.listdir(image_folder)[0])
img = mpimg.imread(img_path)

plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title("Example Chest X-ray")
plt.show()